In [1]:
import pandas as pd
import joblib
import os
import math

# --- Configuração de Caminhos e Limites ---
BASE_DIR = os.getcwd() # Deve ser o diretório 'flight-delay-ds'
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODELS_DIR = os.path.join(BASE_DIR, 'models')
MAX_SIZE_MB = 95 # Margem de segurança abaixo de 100 MB

# ----------------------------------------------------------------------
# 1. FUNÇÃO PARA DIVIDIR DATAFRAMES (PARQUET)
# ----------------------------------------------------------------------

def split_parquet_file(file_name, output_prefix, num_parts=None):
    """Carrega um arquivo Parquet e o divide em partes menores que 95MB."""
    print(f"--- Processando: {file_name} ---")
    input_path = os.path.join(DATA_DIR, file_name)

    # Verifica o tamanho do arquivo em MB
    file_size_mb = os.path.getsize(input_path) / (1024 * 1024)

    if file_size_mb <= MAX_SIZE_MB:
        print(f"Arquivo {file_name} ({file_size_mb:.2f} MB) já está abaixo do limite. Ignorando.")
        return

    df = pd.read_parquet(input_path)
    total_rows = len(df)

    # Calcula o número de partes necessárias com base no tamanho
    if num_parts is None:
        num_parts = math.ceil(file_size_mb / MAX_SIZE_MB)

    chunk_size = total_rows // num_parts
    print(f"Dividindo {file_name} ({total_rows} linhas) em {num_parts} partes de ~{chunk_size} linhas.")

    for i in range(num_parts):
        start = i * chunk_size
        end = (i + 1) * chunk_size if i < num_parts - 1 else total_rows
        chunk = df.iloc[start:end]

        output_path = os.path.join(DATA_DIR, f'{output_prefix}_part_{i+1}.parquet')
        chunk.to_parquet(output_path, index=False)
        print(f"-> Salvo parte {i+1} em: {output_path}. Tamanho: {os.path.getsize(output_path) / (1024 * 1024):.2f} MB")

    # Deletar o arquivo original
    os.remove(input_path)
    print(f"Arquivo original {file_name} deletado.")


# ----------------------------------------------------------------------
# 2. FUNÇÃO PARA DIVIDIR MODELOS (PKL)
# ----------------------------------------------------------------------

def split_pkl_model(file_name, output_prefix):
    """Carrega um modelo .pkl e o divide em partes menores que 95MB."""
    print(f"\n--- Processando Modelo: {file_name} ---")
    input_path = os.path.join(MODELS_DIR, file_name)

    # Verifica o tamanho do arquivo em MB
    file_size_mb = os.path.getsize(input_path) / (1024 * 1024)

    if file_size_mb <= MAX_SIZE_MB:
        print(f"Modelo {file_name} ({file_size_mb:.2f} MB) já está abaixo do limite. Ignorando.")
        return

    # O Joblib é excelente para salvar modelos em partes

    # Calcula o número de partes necessárias (arredondando para cima)
    num_parts = math.ceil(file_size_mb / MAX_SIZE_MB)

    print(f"Dividindo modelo {file_name} ({file_size_mb:.2f} MB) em {num_parts} partes.")

    # A função dump do joblib permite a divisão automática
    output_path = os.path.join(MODELS_DIR, f'{output_prefix}_part')

    joblib.dump(
        joblib.load(input_path), # Carrega o modelo
        output_path,
        compress=3, # Nível de compressão
        protocol=4, # Evita problemas de compatibilidade
        max_bytes=int(MAX_SIZE_MB * 1024 * 1024) # Tamanho máximo em bytes
    )

    # Verifica os arquivos criados
    for f in os.listdir(MODELS_DIR):
        if f.startswith(f'{output_prefix}_part'):
            print(f"-> Salvo parte em: {f}. Tamanho: {os.path.getsize(os.path.join(MODELS_DIR, f)) / (1024 * 1024):.2f} MB")

    # Deletar o arquivo original
    os.remove(input_path)
    print(f"Modelo original {file_name} deletado.")


# ----------------------------------------------------------------------
# 3. EXECUÇÃO PRINCIPAL
# ----------------------------------------------------------------------

# 1. Arquivo de 181 MB (Exige 3 partes para ficar abaixo de 95MB)
split_parquet_file('flight_data_consolidated.parquet', 'consolidated', num_parts=3)

# 2. Arquivo de 105 MB (Exige 2 partes para ficar abaixo de 95MB)
split_parquet_file('flight_data_with_features.parquet', 'features', num_parts=2)

# 3. Modelo de 200 MB (Será dividido automaticamente pelo joblib)
split_pkl_model('random_forest_full_model.pkl', 'rf_model')

print("\n--- Processo de divisão concluído. Arquivos originais deletados. ---")

--- Processando: flight_data_consolidated.parquet ---
Dividindo flight_data_consolidated.parquet (14825727 linhas) em 3 partes de ~4941909 linhas.
-> Salvo parte 1 em: f:\Meus_Projetos\hackathon\FlightOnTime\flight-delay-ds\data\consolidated_part_1.parquet. Tamanho: 33.73 MB
-> Salvo parte 2 em: f:\Meus_Projetos\hackathon\FlightOnTime\flight-delay-ds\data\consolidated_part_2.parquet. Tamanho: 33.75 MB
-> Salvo parte 3 em: f:\Meus_Projetos\hackathon\FlightOnTime\flight-delay-ds\data\consolidated_part_3.parquet. Tamanho: 33.65 MB
Arquivo original flight_data_consolidated.parquet deletado.
--- Processando: flight_data_with_features.parquet ---
Dividindo flight_data_with_features.parquet (11408131 linhas) em 2 partes de ~5704065 linhas.
-> Salvo parte 1 em: f:\Meus_Projetos\hackathon\FlightOnTime\flight-delay-ds\data\features_part_1.parquet. Tamanho: 88.78 MB
-> Salvo parte 2 em: f:\Meus_Projetos\hackathon\FlightOnTime\flight-delay-ds\data\features_part_2.parquet. Tamanho: 97.62 MB
Arquivo

c:\Users\Everton\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Everton\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


TypeError: dump() got an unexpected keyword argument 'max_bytes'